File structure
Folder and files in main directory:
- model.py
- Behavior-Cloning.ipynb
- model.json
- model.h5
- drive.py
- driving_log.csv
- 'IMG' folder
    -- 'All .jpg images'

In [1]:
# Get training dataset from Udacity website

# Imports
from urllib.request import urlretrieve
from os.path import isfile
from tqdm import tqdm
import zipfile
import shutil

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile('driving_log.csv'):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='Train Dataset') as pbar:
        urlretrieve(
            'https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip',
            'data.zip',
            pbar.hook)

    # Unzip data set
    zip_ref = zipfile.ZipFile('data.zip', 'r')
    zip_ref.extractall('')
    zip_ref.close()

    # Move dataset to folder location
    shutil.move('data/IMG/','IMG')
    shutil.move('data/driving_log.csv','driving_log.csv')
    
    print('Training data downloaded.')

In [2]:
# Imports
import pickle
import numpy as np
import math

# Fix error with TF and Keras
import tensorflow as tf
tf.python.control_flow_ops = tf

print('Modules loaded.')

Modules loaded.


In [3]:
# Load all driving log data

# Imports
import csv, sys
import os

filename = 'driving_log.csv'
all_angles = []
imgs_fname = []

with open(filename, 'rt') as csvfile:
    
    # Load driving angle data
    anglereader = csv.reader(csvfile, delimiter=',')

    try:
        for row in anglereader:
            all_angles.append(row[-1])
          
            fln = row[0].split('/')
            imgs_fname.append(fln[-1])
    except csv.Error as e:
        sys.exit('file %s, line %d: %s' % (filename, reader.line_num, e)) 

print('All driving log data loaded.')

All driving log data loaded.


In [4]:
# Load images and driving log data for each batch

# Imports
import os
import os.path
from PIL import Image
import numpy as np

def load_data_batch(batch_num,batch_size):
    
    img_path = os.getcwd()+'/IMG'
    valid_ext = [".jpg"]
    
    i=1
    f_i=1
    imgs = np.empty((0,160,320,3))
    angles = []
    
    for f in os.listdir(img_path)[batch_num*batch_size:]:
        
        f_ext = os.path.splitext(f)[1]
        if f_ext.lower() not in valid_ext:
            continue
            
        if i <= batch_size:
            # First image/angle set
            if f_i == 1:
                img = np.squeeze(Image.open(os.path.join(img_path,f)))
                imgs = np.append(imgs, np.expand_dims(img, axis=0),axis=0)
                
                angles_i = batch_num*batch_size+f_i
                angles.append(all_angles[angles_i])
                
                i+=1
            else:
                angles_i = batch_num*batch_size+f_i
                
                if all_angles[angles_i] not in angles:
                    img = np.squeeze(Image.open(os.path.join(img_path,f)))
                    imgs = np.append(imgs, np.expand_dims(img, axis=0),axis=0)
                    
                    angles_i = batch_num*batch_size+f_i
                    angles.append(all_angles[angles_i])
                    
                    i+=1

        else:
            imgs = imgs.astype(np.uint8)
            return imgs, angles
        
        f_i+=1
    
    return

In [5]:
# Preprocessing functions for image set

# Imports
import cv2
import math

def preprocess_set(img):
    
    img_rsize = resize_set(img)
    img_gr = grayscale_set(img_rsize)
    img_nm = normalize_set(img_gr)
    img_p = np.expand_dims(img_nm, axis=3)
    
    return img_p
    
# Grayscale image set
def grayscale_set(img):
    img_gr = []
    
    for i in range(0,len(img)):
        img_gr.append(cv2.cvtColor(img[i],cv2.COLOR_BGR2GRAY))

    return img_gr

# Normalize image set (values between 0-1)
def normalize_set(img):
    grayscale_max = 255
    img_nm = []
    
    for i in range(0,len(img)):
        img_nm.append(img[i]/grayscale_max)
    
    return img_nm

# Resize single image
def resize_set(img):
    #img resized from (160x320) to (32x64)
    img_height = 32
    img_length = 64
    img_rsize = []
    
    for i in range(0,len(img)):
        img_rsize.append(cv2.resize(img[i],(img_length,img_height)))
    return img_rsize

In [6]:
# Batch processing functions

# Imports
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelBinarizer

def batch_proc(imgs,angles):
    
    # Set up train data
    X_train, y_train = imgs, angles
    
    # Shuffle training image set
    X_train, y_train = shuffle(X_train, y_train)

    # Preprocess training and validation image sets
    X_train = preprocess_set(X_train)
    
    # One Hot encode the labels to the variable y_one_hot
    label_binarizer = LabelBinarizer()
    y_train_one_hot = label_binarizer.fit_transform(y_train)

    return X_train, y_train_one_hot

In [7]:
# Define train and validation batch generators

# Imports
import math
import os
import random

# Parameters
#There will be 43 batches of 56 images each
batch_size = 56
epoch = 10
train_valid_split = 0.8
nb_val_samples = 56

# Total number of images
num_all_img = len(os.listdir(os.getcwd()+'/IMG'))
# Total number of angles data
num_all_ang = len(np.unique(all_angles))
# Total number of set images-angles considered
num_all_data = min(num_all_img,num_all_ang)

# Batch parameters
batch_train_max = math.floor(train_valid_split*num_all_data/batch_size)
batch_validation_max = math.floor(num_all_data/batch_size)
nb_val_samples = math.floor((1-train_valid_split)*num_all_data)

def generate_train_batch(batch_size):
    # Generate data batch
    while 1:
        for batch_num in range (batch_train_max):
            
            imgs, angles = load_data_batch(batch_num,batch_size) 
            X_train, y_train_one_hot = batch_proc(imgs, angles)
 
            if y_train_one_hot.shape[1] == batch_size:
                yield X_train, y_train_one_hot


def generate_validation_batch(batch_size):
    # Generate data batch
    batch_range = batch_validation_max - batch_train_max
    while 1:
        for batch_num in range (batch_range):
            batch_val_num = batch_num + batch_train_max
            
            imgs, angles = load_data_batch(batch_num,batch_size)            
            X_validation, y_validation_one_hot = batch_proc(imgs, angles)
            
            if y_validation_one_hot.shape[1] == batch_size:
                yield X_validation, y_validation_one_hot

In [8]:
# CHOOSE TO IMPORT MODEL AND WEIGHTS
# OPTIONAL: import model.json and model.h5

# Imports
import json
from keras.models import model_from_json

# Load the model from model.json
json_data=open('model.json').read()
json_string = json.loads(json_data)
model = model_from_json(json_string)

#Load the weights from model.h5
model.load_weights('model.h5', by_name=True)

Using TensorFlow backend.


# CHOOSE TO SET UP NEW MODEL AND WEIGHTS

# Network model

# Initialize n_classes
n_classes = batch_size

# Imports
from keras.models import Sequential
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Convolution2D(64, 3, 3, input_shape=(32, 64, 1)))
model.add(MaxPooling2D(((2,2))))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(n_classes))
model.add(Activation('softmax'))

In [9]:
# Compile and train the model
model.compile('adam', 'categorical_crossentropy', ['accuracy'])

history = model.fit_generator(generate_train_batch(batch_size),
                              samples_per_epoch = num_all_data, nb_epoch = epoch, 
                              validation_data = generate_validation_batch(batch_size),
                              nb_val_samples = nb_val_samples, verbose = 1, initial_epoch = 0)

Epoch 1/10
2408/2408 [==============================] - 57s - loss: 3.8461 - acc: 0.0536 - val_loss: 3.7990 - val_acc: 0.0556

In [10]:
# Save the model and weights

# Imports
from keras.models import model_from_json
import json

# Save the model to model.json
json_string = model.to_json()
with open('model.json', 'w') as f:
     json.dump(json_string, f)

# Save the weights to model.h5
model.save_weights('model.h5')

In [ ]:
def display_info():
    # Summary of Dataset information
    # Shape of an traffic sign image
    image_shape = X_train[0].shape
    # Number of training examples
    n_train = X_train.shape[0]
    # Number of unique classes/labels there are in the dataset
    n_classes = len(np.unique(y_train))
    # Display information
    print("Number of training examples total = ", num_all_img)
    print("Image data shape =", image_shape)
    print("Number of training examples in batch =", n_train)
    print("Number of classes in batch =", n_classes)